In [1]:
import sys

sys.path.append('..')

from src_refactored.configs import DataConfig, NetConfig, NetArchitecture, FilterConfig
from src_refactored.utils import train, get_wandb_logger
from src_refactored.configs import PACKAGE_PATH
from src_refactored.data.data_processor import DataProcessor

# DataLoader

In [2]:
filter=FilterConfig(
    n_bins= 30,
    n_gaps= 10,
    gap_size= 5, 
    rms_ratio= 0.,
    non_zero_ratio=0.8
)

data_config = DataConfig(
    path="/mnt/c/work/Fall_2021_csv",
    output_path=f"{PACKAGE_PATH}/resources/datasets",
    class_names=["cz_3", "falcon_9", "atlas_V",  "h2a", "globalstar"],
    regexes=[r'CZ-3B.*', r'FALCON_9.*', r'ATLAS_[5|V]_CENTAUR_R_B$',  r'H-2A.*', r'GLOBALSTAR.*'],
    validation_split=0.2,
    split_strategy="objectID",
    number_of_training_examples_per_class=10_000,
    filter_config=filter,
    convert_to_mag=False,
    fourier=True,
    std= False,
    rms= False,
    residuals= False,
    amplitude= False,
    lc= False,
    reconstructed_lc= False,
    push_to_max= False
)

In [3]:
dp = DataProcessor(data_config)
dp.load_data_MMT()
# dp.load_raw_data_MMT()
# dp.save_data_MMT()

In [4]:
train_set, val_set = dp.get_pytorch_datasets()

# Lightning module

In [9]:
import sys
sys.path.append('..')

from src_refactored.configs import NetConfig, NetArchitecture, FCConfig, CNNConfig, CNNFCConfig, PACKAGE_PATH
from src_refactored.configs import ModelConfig
from src_refactored.module.fc import FC
from src_refactored.module.cnn import CNN
from src_refactored.module.cnnfc import CNNFC
from src_refactored.module.lightning_module import LCModule

from src_refactored.utils import train, get_wandb_logger

In [6]:
net_cfg = NetConfig(
    name="default",
    input_size=16,
    output_size=5,
    architecture=NetArchitecture.FC,
    args=FCConfig(input_size=16, output_size=5, layers=[])  
)

In [7]:
module = LCModule(net_cfg)


In [11]:
logger = get_wandb_logger("LC")
train(module, dp, 2, 32,4, logger=logger)


wandb: Currently logged in as: dano-kyselica. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/poirot/.netrc


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type             | Params
-----------------------------------------------
0 | net       | FC               | 85    
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
85        Trainable params
0         Non-trainable params
85        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▁▁▁███████
train_acc_epoch,▁█
train_acc_step,▁▁▁▁▁▁▁▁▁▁
train_loss_epoch,█▁
train_loss_step,█▆▆▇▄▃▁▂▁▂
trainer/global_step,▂▄▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▆▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃█
val_acc_epoch,▁▁
val_acc_step,█████████████████████████▁██████████████
val_loss_epoch,█▁
val_loss_step,▆█▆▆▅▇█▄▅▃▇▆▆▅▆▄█▃▃▆▄▆▅▃▃▄▅▁▃▃▂▃▃▂▂▄▄▃▄▃
epoch,1
